In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import re
import math

In [2]:
df = pd.read_csv("raw_data.csv", sep='\t')
df.head()

,Unnamed: 0,Link,Tên,Loại nhà,Diện tích,Các loại phòng,Giá,Vị trí,Tiện ích
0,0,https://www.luxstay.com/vi/rooms/13967,JOLIE HOUSE - BIỆT THỰ HƯỚNG RA VƯỜN - CÁCH TR...,Biệt Thự,100 m2,Nguyên căn · 5 phòng tắm · 6 giường · 4 phòng ...,"2,887,500₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Đệm bổ s..."
1,1,https://www.luxstay.com/vi/rooms/33714,Gold&Ruby House- Phòng Quadruple,Căn hộ dịch vụ,20 m2,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"600,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú..."
2,2,https://www.luxstay.com/vi/rooms/15456,Maika Condotel - Triple Bedroom,Căn hộ chung cư,110 m2,Nguyên căn · 2 phòng tắm · 3 giường · 3 phòng ...,"2,750,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."
3,3,https://www.luxstay.com/vi/rooms/18878,Vườn đom đóm Đà Lạt- Phòng đơn hướng vườn rộng...,Nhà riêng,12 m2,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"370,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp..."
4,4,https://www.luxstay.com/vi/rooms/15377,Rebeka's Home 1 - Góc vườn nhỏ tĩnh lặng trước...,Nhà riêng,15 m2,Phòng riêng · 1 phòng tắm · 1 giường · 1 phòng...,"350,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."


# Chỉnh sửa dữ liệu
Trước tiên ta cần format lại data để tiện làm việc hơn

## Đặt lại tên các cột để thuận tiện trong việc truy cập

In [3]:
df.columns = ["link", "name", "type", "area", "room_types", "price", "location", "other"]
df.head()

ValueError: Length mismatch: Expected axis has 9 elements, new values have 8 elements

## Chỉnh sửa kiểu dữ liệu của các thuộc tính

Trước tiên ta xem xét kiểu dữ liệu của các cột trong dữ liệu:

In [ ]:
df.dtypes

Có thể thấy kiểu dữ liệu của tất cả các cột đều là "object". Tuy nhiên, để có thể áp dụng các thuật toán máy học lên dữ liệu, ta cần đưa dữ liệu của một số thuộc tính về dạng số. Cụ thể, trước tiên ta sẽ format lại kiểu dữ liệu của các thuộc tính "type", "area" và "price"

### Type

"Type" là loại phòng. Ta có thể chuyển đổi thuộc tính này sang dạng sô bằng cách gán một id riêng biệt cho mỗi loại phòng.

Đầu tiên, ta xem số lượng phòng khác nhau:

In [ ]:
df["type"].describe()

Như vậy, ta có 10 loại phòng và loại phòng xuất hiện nhiều nhất là "Căn hộ chung cư"

Kế đến, ta đếm số lượng môi loại phòng:

In [ ]:
df["type"].value_counts()

Có thể thấy trong 10 loại phòng trên thì chỉ có 6 loại phòng là đúng với thực tế, còn lại là các giá trị lỗi sinh ra trong quá trình thu thập dữ liệu. Ta sẽ tiến hành loại các giá trị lỗi đó trước:

In [ ]:
counts = dict(df["type"].value_counts())
real_types = list(df["type"].value_counts().keys())
real_types = [x for x in real_types if counts[x] > 1]
real_types

In [ ]:
df = df[df["type"].apply(lambda x: x in real_types)]

Kiểm tra lại ta sẽ thấy các giá trị lỗi đã bị loại bỏ:

In [ ]:
df["type"].value_counts()

In [ ]:
fig = plt.figure(figsize=(9, 4))
sns.countplot(x="type", data=df)
plt.show()

Để thuận lợi cho việc tra cứu lại, ta sẽ tạo một <i>dictionary</i> lưu lại việc mapping các loại phòng thành các id:

In [ ]:
type_id = {}
for x in range(len(real_types)):
    type_id[real_types[x]] = x
type_id

Map các loại phòng thành các giá trị số và kiểm tra lại:

In [ ]:
df["type"] = df["type"].apply(lambda x: type_id[x])

In [ ]:
fig = plt.figure(figsize=(9, 4))
sns.countplot(x="type", data=df)
plt.show()

Kiểm tra lại và ta sẽ thấy kiểu dữ liệu của "type" đã được chuyển sang dạng int64

In [ ]:
df.dtypes

### Area

Dựa vào quan sát, ta thấy chỉ cần loại "m2" trong các giá trị của thuộc tính là đủ

In [ ]:
new_area = df["area"].apply(lambda x: float(str(x).split(' ')[0]))
df["area"] = new_area
new_area

Sau đó kiểm tra lại kiểu dữ liệu của "area":

In [ ]:
df["area"].describe()

### Price

Ta sử dụng regular regression để lọc ra giá tiền:

In [ ]:
new_price = df["price"].apply(lambda x: re.search("[0-9,]*", str(x)).group().replace(',', ''))
new_price = new_price.apply(lambda x: float(x) if x != "" else float('nan'))
df["price"] = new_price

Sau đó kiểm tra lại kiểu dữ liệu của "price":

In [ ]:
df["price"].describe()

## Chọn lựa thuộc tính và tiền xử lý dữ liệu
Ta sẽ tìm hiểu và chọn ra các thuộc tính có ích để áp dụng máy học, đồng thời đưa dữ liệu về dạng giàu thông tin hơn

Trước tiên, ta tạo một hàm để đếm số mẫu dữ liệu bị thiếu đối với một thuộc tính:

In [ ]:
def count_missing(df, col):
    print(col, df[col].isnull().sum())

### link
Ta dễ dàng thấy đường dẫn của một phòng cho thuê không hề liên quan đến giá của phòng đó, do đó ta chỉ đơn giản bỏ thuộc tính đó

In [ ]:
df = df.drop("link",axis=1)
df.head()

### type
Đầu tiên, ta kiểm tra số mẫu dữ liệu bị thiếu:

In [ ]:
count_missing(df, "type")

Như vậy, thuộc tính "type" không có dữ liệu thiếu. Ta tiếp tục xem xét mối quan hệ giữa loại phòng và giá phòng:

In [ ]:
fig = plt.figure(figsize=(10,7))
sns.boxplot(x="type", y="price", data=df)
plt.show()

Ta có thể thấy sự ảnh hưởng của loại nhà lên giá tiền. Cụ thể, ta thấy sự ảnh hưởng trên tăng dần với thứ tự 3, 2, 1, 0 và 5. Đặc biệt, loại nhà có id là 5 cho thấy giá của nó cao hơn hẳn so với các loại nhà khác. Xem lại cách đánh số các loại nhà, ta có thứ tự sau: "Căn hộ studio" -> "Nhà riêng" -> "Căn hộ dịch vụ" -> "Căn hộ chung cư" -> "Biệt thự". Như vậy, ta sẽ đặt giá trị của thuộc tính "type" lại sao cho giá trị ấy thể hiện thứ tự này.

Đối với loại "khác" (4), ta không rõ loại nhà này ảnh hưởng như thế nào đến giá thuê nên ta sẽ đặt một biến riêng để xác định các mẫu với loại phòng này là "không biết".

In [ ]:
type_id

In [ ]:
id_to_type = {}
for x in type_id:
    id_to_type[type_id[x]] = x
new_type_id = {"Khác":0, "Căn hộ Studio":1, "Nhà riêng":2, "Căn hộ dịch vụ":3, "Căn hộ chung cư":4, "Biệt Thự":5}
df["type"] = df["type"].apply(lambda x: new_type_id[id_to_type[x]])

In [ ]:
fig = plt.figure(figsize=(10,7))
sns.boxplot(x="type", y="price", data=df)
plt.show()

### Area
Ta bắt đầu bằng việc kiểm tra dữ liệu bị thiếu:

In [ ]:
count_missing(df, "area")

Có 121 mẫu dữ liệu bị thiếu diện tích. Ta có thể bỏ các mẫu dữ liệu trên hoặc điền vào những chỗ trống bằng cách sử dụng  giá trị trung bình. Ở đây ta sẽ sử dụng phương pháp điền giá trị trung bình. Tuy nhiên, trước tiên, ta cần lọc bớt các outlier để hạn chế sự ảnh hưởng của chúng lên giá trị trung bình.

In [ ]:
df["area"].describe()

Ta có thể thấy giá trị nhỏ nhất là $2$ và lớn nhất là $10000$. Trên thực tế, hiếm khi có trường hợp trên xảy ra. Để tránh việc giá trị trung bình bị các yếu tố nhiễu ảnh hưởng, ta trước tiên cần lọc bớt các outlier. Để làm điều đó, ta vẽ biểu đồ phân bố của dữ liệu:

In [ ]:
tmp_area = df[df["area"].isnull() == False]["area"]
fig = plt.figure(figsize=(10,7))
sns.distplot(tmp_area, bins=100)
plt.show()

Thoạt nhìn, ta sẽ lọc bớt các mẫu dữ liệu với diện tích lớn hơn $2000$, sau đó quan sát lại biểu đồ.

In [ ]:
tmp_area = df[(df["area"].isnull() == False) & (df["area"] <= 2000)]["area"]
fig = plt.figure(figsize=(10,7))
sns.distplot(tmp_area, bins=100)
plt.show()

In [ ]:
tmp_area = df[(df["area"].isnull() == False) & (df["area"] <= 200)]["area"]
fig = plt.figure(figsize=(10,7))
sns.distplot(tmp_area, bins=50)
plt.show()

In [ ]:
df[df["area"] >= 2000].head(10)

In [ ]:
tmp_area.describe()

In [ ]:
df = df[df["area"].isnull() | ((df["area"] >= 5) & (df["area"] <= 500))]
df.shape

In [ ]:
df["area"].describe()

Sau khi loại xong các outlier trên, dựa vào quy tắc $99%$ dữ liệu nằm trong $3$ bước của độ lệch chuẩn, ta tiếp tục loại các mẫu dữ liệu có diện tích lớn hơn $300$.

In [ ]:
df = df[df["area"].isnull() | (df["area"] <= 300)]
df.shape

In [ ]:
df["area"].describe()

In [ ]:
tmp = df.groupby("type").mean()["area"]
tmp

In [ ]:
new_area = df["area"].apply(lambda x: x if math.isnan(x) == False else df["area"].mean())
sns.distplot(new_area)

In [ ]:
fig = plt.figure(figsize=(10,7))
tmp_df = df[df["area"].isnull() == False]
sns.boxplot(x="type", y="area", data=tmp_df)